<h1>Stacked Auto Encoders Training 7-13-2016</h1>

Assuming CNN-H1, and the 6 CNNs for the patches have been trained, we can use the weights to get activations for their fc6 layer and train 3 layers of our SAE individually.

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%matplotlib inline

import theano.sandbox.cuda
theano.sandbox.cuda.use('gpu0')

import os
import numpy as np
import matplotlib.pyplot as plt

from multiprocessing import Process
from scipy.ndimage.filters import gaussian_filter

from skimage import io
from skimage.color import rgb2grey
from skimage.transform import resize

from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers

# from keras.utils import np_utils

np.random.seed(123456)

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 98.0% of memory, CuDNN 3007)


In [3]:
cnn_h1_weight_file = 'CNN-H1_epoch-20_activations'
cnn_p1_weight_file = 'CNN-P1_epoch-20_activations'
cnn_p2_weight_file = 'CNN-P2_epoch-20_activations'
cnn_p3_weight_file = 'CNN-P3_epoch-20_activations'
cnn_p4_weight_file = 'CNN-P4_epoch-20_activations'
cnn_p5_weight_file = 'CNN-P5_epoch-20_activations'
cnn_p6_weight_file = 'CNN-P6_epoch-20_activations'
y_train_file = 'y_train_activations'

layer_1_size = 2048
layer_2_size = 1024
layer_3_size = 512

validation_split = .15


In [4]:
def load_activations(file_name):
    return np.load('models/' + file_name + '.npy')

In [5]:
cnn_h1_activations = load_activations(cnn_h1_weight_file)
cnn_p1_activations = load_activations(cnn_p1_weight_file)
cnn_p2_activations = load_activations(cnn_p2_weight_file)
cnn_p3_activations = load_activations(cnn_p3_weight_file)
cnn_p4_activations = load_activations(cnn_p4_weight_file)
cnn_p5_activations = load_activations(cnn_p5_weight_file)
cnn_p6_activations = load_activations(cnn_p6_weight_file)
y_train = load_activations(y_train_file)

In [6]:
def fix_activations(_activations):
    return np.array([activation for activations in _activations for activation in activations])

In [7]:
cnn_h1_activations = fix_activations(cnn_h1_activations)
cnn_p1_activations = fix_activations(cnn_p1_activations)
cnn_p2_activations = fix_activations(cnn_p2_activations)
cnn_p3_activations = fix_activations(cnn_p3_activations)
cnn_p4_activations = fix_activations(cnn_p4_activations)
cnn_p5_activations = fix_activations(cnn_p5_activations)
cnn_p6_activations = fix_activations(cnn_p6_activations)

In [8]:
cnn_h1_activations.shape

(26668, 512)

In [9]:
activations = np.concatenate((
        cnn_h1_activations, 
        cnn_p1_activations,
        cnn_p2_activations,
        cnn_p3_activations,
        cnn_p4_activations,
        cnn_p5_activations,
        cnn_p6_activations), axis=1)

In [10]:
activations.shape

(26668, 3584)

In [11]:
y_train.shape

(26668,)

<h2>Auto Encoder</h2>

In [12]:
def get_encoder(X_train, X_test, encoding_dim, nb_epoch=50, regularizer=10e-5):
    assert X_train.shape[1] == X_test.shape[1]
    shape = X_train.shape[1]

    input_img = Input(shape=(shape,))
    encoded = Dense(
        encoding_dim, 
        activation='relu', 
        activity_regularizer=regularizers.activity_l2(regularizer)
    )(input_img)
    decoded = Dense(shape, activation='sigmoid')(encoded)

    autoencoder = Model(input=input_img, output=decoded)
    encoder = Model(input=input_img, output=encoded)

    # sgd = SGD(lr=.01, decay=99.9) #lr=0.01->0.00001
    autoencoder.compile(
        optimizer='adadelta', 
        loss='categorical_crossentropy', 
        metrics=['accuracy'])
    
    autoencoder.fit(X_train, X_train,
                nb_epoch=nb_epoch,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))
    return autoencoder, encoder
    

In [13]:
x_train = activations[:int(activations.shape[0] * (1 - validation_split))]
x_test = activations[int(activations.shape[0] * (1 - validation_split)):]
X_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
X_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [14]:
autoencoder1, encoder1 = get_encoder(X_train, X_test, 2048, 30, 10e-3)

Train on 22667 samples, validate on 4001 samples
Epoch 1/30
22667/22667 [==============================] - 4s - loss: -57218.2613 - acc: 0.0609 - val_loss: -66095.6572 - val_acc: 0.1027
Epoch 2/30
22667/22667 [==============================] - 4s - loss: -66930.3240 - acc: 0.1424 - val_loss: -68691.4351 - val_acc: 0.1612
Epoch 3/30
22667/22667 [==============================] - 4s - loss: -69162.9925 - acc: 0.1875 - val_loss: -70336.9225 - val_acc: 0.1857
Epoch 4/30
22667/22667 [==============================] - 4s - loss: -70396.5857 - acc: 0.2150 - val_loss: -71400.6475 - val_acc: 0.2219
Epoch 5/30
22667/22667 [==============================] - 4s - loss: -71273.8733 - acc: 0.2389 - val_loss: -71893.4412 - val_acc: 0.2447
Epoch 6/30
22667/22667 [==============================] - 4s - loss: -71922.7869 - acc: 0.2578 - val_loss: -72551.2002 - val_acc: 0.2537
Epoch 7/30
22667/22667 [==============================] - 4s - loss: -72390.3669 - acc: 0.2709 - val_loss: -72801.4965 - val_acc:

In [15]:
X_train = encoder1.predict(X_train)
X_test = encoder1.predict(X_test)

print X_train.shape
print X_test.shape

/usr/local/lib/python2.7/dist-packages/keras/backend/theano_backend.py:514: UserWarning: theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 1 is not part of the computational graph needed to compute the outputs: keras_learning_phase.
To make this warning into an error, you can pass the parameter on_unused_input='raise' to theano.function. To disable it completely, use on_unused_input='ignore'.
  **kwargs)


(22667, 2048)
(4001, 2048)


In [16]:
autoencoder2, encoder2 = get_encoder(X_train, X_test, 1024, 50)

Train on 22667 samples, validate on 4001 samples
Epoch 1/50
22667/22667 [==============================] - 2s - loss: 41507.4702 - acc: 0.0097 - val_loss: 40334.3564 - val_acc: 0.0575
Epoch 2/50
22667/22667 [==============================] - 2s - loss: 40034.2114 - acc: 0.1081 - val_loss: 39333.6829 - val_acc: 0.1515
Epoch 3/50
22667/22667 [==============================] - 2s - loss: 39175.0917 - acc: 0.1829 - val_loss: 38768.4483 - val_acc: 0.2052
Epoch 4/50
22667/22667 [==============================] - 2s - loss: 38576.8261 - acc: 0.2334 - val_loss: 38230.3148 - val_acc: 0.2599
Epoch 5/50
22667/22667 [==============================] - 2s - loss: 38140.7666 - acc: 0.2760 - val_loss: 38006.1731 - val_acc: 0.3002
Epoch 6/50
22667/22667 [==============================] - 2s - loss: 37807.2105 - acc: 0.3141 - val_loss: 37669.6898 - val_acc: 0.3289
Epoch 7/50
22667/22667 [==============================] - 2s - loss: 37562.8501 - acc: 0.3386 - val_loss: 37510.3859 - val_acc: 0.3502
Epoch 

In [17]:
X_train = encoder2.predict(X_train)
X_test = encoder2.predict(X_test)

print X_train.shape
print X_test.shape

(22667, 1024)
(4001, 1024)


In [18]:
autoencoder3, encoder3 = get_encoder(X_train, X_test, 512, 50)

Train on 22667 samples, validate on 4001 samples
Epoch 1/50
22667/22667 [==============================] - 1s - loss: 15102.8008 - acc: 0.0023 - val_loss: 14471.7531 - val_acc: 0.0037
Epoch 2/50
22667/22667 [==============================] - 1s - loss: 14662.7052 - acc: 0.0079 - val_loss: 14155.3762 - val_acc: 0.0167
Epoch 3/50
22667/22667 [==============================] - 1s - loss: 14318.1233 - acc: 0.1549 - val_loss: 13848.2065 - val_acc: 0.2659
Epoch 4/50
22667/22667 [==============================] - 1s - loss: 14035.5679 - acc: 0.2816 - val_loss: 13670.9940 - val_acc: 0.3114
Epoch 5/50
22667/22667 [==============================] - 1s - loss: 13839.5515 - acc: 0.3396 - val_loss: 13501.9122 - val_acc: 0.3667
Epoch 6/50
22667/22667 [==============================] - 1s - loss: 13695.6872 - acc: 0.3725 - val_loss: 13397.1453 - val_acc: 0.3949
Epoch 7/50
22667/22667 [==============================] - 1s - loss: 13585.0776 - acc: 0.3987 - val_loss: 13313.9699 - val_acc: 0.4401
Epoch 

In [19]:
X_train = encoder3.predict(X_train)
X_test = encoder3.predict(X_test)

print X_train.shape
print X_test.shape

(22667, 512)
(4001, 512)
